# CRM Data Quality Audit – Demo 01

**Project Objective:**  
Evaluate the structure, completeness, and usability of a real-world CRM export dataset using Python and Pandas.

**Business Value:**  
A clean CRM enables better segmentation, customer retention strategies, and accurate reporting.

---

**Summary Metrics:**

| Metric                     | Value        |
|----------------------------|--------------|
| Total Records              | 541,909      |
| Total Customers            | 4,372        |
| Missing Customer IDs       | 135,080 (25%) |
| Duplicate Rows             | 5268         |
| Countries Covered          | 38           |
| Negative Quantities        | 10624        |
| Data Quality Score         | 73.6         |


___________________________________________________________________________________________________________________

### **Import libraries**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from IPython.display import display

___________________________________________________________________________________________________________________

### **Import Dataset**

In [ ]:
# Load data
df = pd.read_excel("/Users/jacquelinehardy/Desktop/jh-data-studio-demos/demo-01-crm-audit/data/Online Retail.xlsx")

# Quick look
df.head()

## Data Dictionary

| Column       | Description                                        |
|--------------|----------------------------------------------------|
| InvoiceNo    | Unique invoice number                              |
| StockCode    | Product identifier                                 |
| Description  | Name of the product                                |
| Quantity     | Number of units purchased (negative = return)      |
| InvoiceDate  | Timestamp of transaction                           |
| UnitPrice    | Price per unit                                     |
| CustomerID   | Unique customer identifier                         |
| Country      | Country of the purchasing customer                 |


___________________________________________________________________________________________________________________

### **Initial Overview**

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
summary = pd.DataFrame({
    "Issue": ["Missing Customer IDs", "Duplicate Rows", "Negative Quantities"],
    "Count": [
        df['CustomerID'].isnull().sum(),
        df.duplicated().sum(),
        (df['Quantity'] < 0).sum()
    ]
})
summary

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
missing_pct = df.isnull().mean().sort_values(ascending=False)
missing_pct[missing_pct > 0].plot(kind='barh', figsize=(10, 4), title="Missing Data by Column")

In [ ]:
sns.boxplot(df['UnitPrice'])
plt.title("Boxplot of Unit Price – Outlier Check")

In [ ]:
outliers = df[df['UnitPrice'] > 1000].sort_values(by='UnitPrice', ascending=False)
outliers[['InvoiceNo', 'Description', 'Quantity', 'UnitPrice', 'CustomerID']].head(5)

In [ ]:
score = 100
score -= df['CustomerID'].isnull().mean() * 100
score -= df.duplicated().mean() * 50
score -= (df['Quantity'] < 0).mean() * 50
score = round(score, 1)

print(f"Data Quality Score: {score}/100")


In [ ]:
plt.barh(["Quality Score"], [score], color='green')
plt.xlim(0, 100)
plt.title("Overall Data Quality")
plt.show()

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df['Country'].value_counts().head(10)

In [ ]:
df['CustomerID'].nunique()

In [ ]:
df['Month'] = df['InvoiceDate'].dt.to_period('M')
df.groupby('Month')['InvoiceNo'].nunique().plot(kind='bar', figsize=(12, 4))

## Cleaning Recommendations

- Remove or impute missing `CustomerID`s (depending on use case)
- Drop 5000+ exact duplicate records
- Remove negative quantities (returned items should be separate)
- Normalize country and product descriptions


In [ ]:
df_clean = df.dropna(subset=['CustomerID'])
df_clean = df_clean[df_clean['Quantity'] >= 0]
df_clean = df_clean.drop_duplicates()

In [ ]:
excluded_rows = df.shape[0] - df_clean.shape[0]
excluded_pct = round((excluded_rows / df.shape[0]) * 100, 2)
print(f"{excluded_rows} rows removed ({excluded_pct}% of original data).")

In [ ]:
df_clean['Total'] = df_clean['Quantity'] * df_clean['UnitPrice']

In [ ]:
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (df_clean['InvoiceDate'].max() - x.max()).days,
    'InvoiceNo': 'nunique',
    'Total': 'sum'
}).rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'Total': 'Monetary'
})

rfm.describe()


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 4))
rfm['Recency'].plot(kind='hist', bins=30, ax=ax[0], title='Recency')
rfm['Frequency'].plot(kind='hist', bins=30, ax=ax[1], title='Frequency')
rfm['Monetary'].plot(kind='hist', bins=30, ax=ax[2], title='Monetary Value')
plt.tight_layout()
plt.show()

In [ ]:
# Keep original for reference
rfm_original = rfm.copy()

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

inertia = []

# Try different values for k
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()


In [ ]:
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(rfm_scaled)
    score = silhouette_score(rfm_scaled, labels)
    print(f"Silhouette Score for k={k}: {round(score, 3)}")

**Elbow Chart Insight**
- The “elbow” clearly appears at k = 3 or 4

- After 4, the inertia continues dropping — but the gain flattens, meaning additional clusters add little value

- This suggests k = 3 or 4 are both efficient choices

**Silhouette Scores**
- k = 2 has the highest score -> too simple
- k = 4 
    - Strong score
    - Clear elbow support from above 
    - Grouping as loyal/at risk/new/bargain
    

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm['Segment'] = kmeans.fit_predict(rfm_scaled)


In [ ]:
rfm['Segment'] = kmeans.labels_

segment_summary = rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Segment': 'count'
}).rename(columns={'Segment': 'Customer Count'})

segment_summary.style.background_gradient(cmap='Blues')

In [ ]:
segment_labels = {
    0: "At Risk",
    1: "VIP",
    2: "Inactive",
    3: "Loyal"
}

rfm['SegmentLabel'] = rfm['Segment'].map(segment_labels)

In [ ]:
rfm['SegmentLabel'].value_counts().plot(kind='bar', color='skyblue', title="Customer Count per Segment")
plt.ylabel("Number of Customers")
plt.xlabel("Segment")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


## Segment Strategy Recommendations

| Segment   | Description                                                       | Strategy                                      |
|-----------|-------------------------------------------------------------------|-----------------------------------------------|
| **VIP**       | Extremely active, very frequent, high spend – your best customers | Offer exclusive perks, early access, VIP loyalty rewards |
| **Loyal**     | Active and high-spending, good frequency                      | Maintain engagement, surprise discounts, referral incentives |
| **At Risk**   | Large group, moderate value and frequency, not recent          | Send reactivation emails, win-back offers, ask for feedback |
| **Inactive**  | Long gone, low engagement, minimal contribution                | Suppress from active campaigns, retarget if they were once high-value |


In [ ]:
df_clean.to_csv("../data/crm_cleaned.csv", index=False)